# xgBoost

In [7]:
import pandas as pd
import numpy as np 
import os 

CLEAN_DATA_DIR = "../data/clean/"

In [8]:
from sklearn.model_selection import train_test_split

data = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_UNFILLED.csv"))
X_train, X_test, y_train, y_test = train_test_split(data.drop(['psu_hh_idcode', 'hhid', 'subjectivePoverty_rating'], axis='columns'), data['subjectivePoverty_rating'], test_size=0.2, random_state=101)


y_train = y_train - 1
y_test = y_test - 1

In [9]:
from sklearn.model_selection import train_test_split

data = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_UNFILLED.csv"))
X_train, X_test, y_train, y_test = train_test_split(data.drop(['psu_hh_idcode', 'hhid', 'subjectivePoverty_rating'], axis='columns'), data['subjectivePoverty_rating'], test_size=0.2, random_state=101)

y_train = y_train - 1
y_test = y_test - 1

from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [1, 3, 5, 7],
    'n_estimators': [50, 100, 200, 500],
    'subsample': [0.3, 0.5, 0.7, 0.9],
    'colsample_bytree': [0.4, 0.6, 0.8, 1.0]
}

# Create the XGBoost model
xgb_model = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', random_state=101)

# Create the GridSearchCV object
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_log_loss',  # Use log loss as the evaluation metric
    cv=5,                    
    verbose=1,               
    n_jobs=-1                
)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Log Loss Score:", -grid_search.best_score_)

best_model = grid_search.best_estimator_
preds = best_model.predict(X_test)
preds_proba = best_model.predict_proba(X_test)

# Evaluate log loss on the test data
from sklearn.metrics import log_loss
print(f"\nLog Loss from test: {log_loss(y_test, preds_proba)}")

Fitting 5 folds for each of 1024 candidates, totalling 5120 fits
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 1, 'n_estimators': 200, 'subsample': 0.3}
Best Log Loss Score: 1.9336183149867185

Log Loss from test: 1.9665212339384595


In [13]:
# Load the dataset
data = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_UNFILLED.csv"))
id_column = data['psu_hh_idcode']

# Prepare the training and testing datasets
                    
# X_train, X_test, y_train, y_test, test_ids = train_test_split(
#     data.drop(['psu_hh_idcode', 'hhid', 'subjectivePoverty_rating'], axis='columns'),
#     data['subjectivePoverty_rating'],
#     id_column,
#     test_size=0,
#     random_state=101
# )

# Use the entire dataset for training
X_train = data.drop(['psu_hh_idcode', 'hhid', 'subjectivePoverty_rating'], axis='columns')
y_train = data['subjectivePoverty_rating'] - 1 # Adjust the target variable to start from 0 (for multi-class classification)






# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [1, 3, 5, 7],
    'n_estimators': [50, 100, 200, 500],
    'subsample': [0.3, 0.5, 0.7, 0.9],
    'colsample_bytree': [0.4, 0.6, 0.8, 1.0]
}

# Create the XGBoost model
xgb_model = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', random_state=101)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_log_loss',  # Use log loss as the evaluation metric
    cv=5,
    verbose=1,
    n_jobs=-1
)

# Perform gridsearch to find the best model
grid_search.fit(X_train, y_train)

# Get the best model and its parameters
print("Best Parameters:", grid_search.best_params_)
print("Best Log Loss Score:", -grid_search.best_score_)

best_model = grid_search.best_estimator_

# Predicting on actual test data

test_data = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TEST_INPUT.csv"))
test_X = test_data.drop(columns=['psu_hh_idcode'])
test_ids = test_data['psu_hh_idcode']

# reorder cols in teest_dat for alignment with training data

# test_data.to_csv(os.path.join(CLEAN_DATA_DIR, TE))
# Predict probabilities for the test dataset
preds_proba = best_model.predict_proba(test_X)

# Evaluate log loss on the test data
# print(f"\nLog Loss from test: {log_loss(y_test, preds_proba)}")

# Create the output DataFrame
output_df = pd.DataFrame(preds_proba, columns=[f'subjectivePoverty_rating_{i+1}' for i in range(preds_proba.shape[1])])
output_df.insert(0, 'psu_hh_idcode', test_ids.values)  # Insert the ID column at the start

# Save to CSV
output_file = "test_predictions.csv"
output_df.head()
# output_df.to_csv(output_file, index=False)
# print(f"Predictions saved to {output_file}")


Fitting 5 folds for each of 1024 candidates, totalling 5120 fits
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 1, 'n_estimators': 200, 'subsample': 0.3}
Best Log Loss Score: 1.9381333016168707


ValueError: feature_names mismatch: ['q02', 'q03', 'q05', 'q09', 'q23', 'Q01', 'Q03', 'Q06', 'Q07', 'Q08', 'Q11', 'Q19'] ['q02', 'q03', 'q09', 'q05', 'q23', 'Q01', 'Q03', 'Q06', 'Q07', 'Q08', 'Q11', 'Q19']

Baseline log loss = $-log(\frac{1}{\#classes}) = -log(\frac{1}{10}) = 2.303$

In [ ]:
print("unique values in test classes:", sorted(list(y_test.unique())))
print("unique values in predicted classes:", sorted(list(np.unique(preds))))

In [6]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming y_test is a Pandas Series

# Calculate the proportion of each value
value_counts = y_test.value_counts(normalize=True).sort_index() 
print(value_counts)

subjectivePoverty_rating
0    0.035614
1    0.087160
2    0.164948
3    0.192127
4    0.206186
5    0.157451
6    0.086223
7    0.057170
8    0.010309
9    0.002812
Name: proportion, dtype: float64
